In [2]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

def preprocess_data(image, label):
    image = tf.image.resize(image, (32, 32))
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, tf.one_hot(label, depth=10)

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

(ds_train, ds_test), ds_info = tfds.load("cifar10", split=["train", "test"], as_supervised=True, with_info=True)

BATCH_SIZE = 64
AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_train = ds_train.map(preprocess_data, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.map(lambda x, y: (data_augmentation(x, training=True), y))
ds_train = ds_train.shuffle(5000).batch(BATCH_SIZE).prefetch(AUTOTUNE)

ds_test = ds_test.map(preprocess_data, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(AUTOTUNE)

base_model = tf.keras.applications.MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights="imagenet")

for layer in base_model.layers[-30:]:
    layer.trainable = True

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation="softmax")
])

initial_learning_rate = 0.0005
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=1000, decay_rate=0.9)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=["accuracy"])

EPOCHS = 20
model.fit(ds_train, validation_data=ds_test, epochs=EPOCHS)

model.save("cifar10_mobilenetv2_finetuned.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

tflite_model_path = "cifar10_mobilenetv2_finetuned.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved at {tflite_model_path}")


<ipython-input-2-11b72513c4e3>:28: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights="imagenet")


Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 154s 126ms/step - accuracy: 0.3120 - loss: 2.1227 - val_accuracy: 0.3536 - val_loss: 1.8329
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 87s 82ms/step - accuracy: 0.5842 - loss: 1.2263 - val_accuracy: 0.5744 - val_loss: 1.2202
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 69s 82ms/step - accuracy: 0.6484 - loss: 1.0383 - val_accuracy: 0.6881 - val_loss: 0.8862
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 72s 85ms/step - accuracy: 0.6879 - loss: 0.9166 - val_accuracy: 0.7106 - val_loss: 0.8280
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 68s 81ms/step - accuracy: 0.7142 - loss: 0.8463 - val_accuracy: 0.7070 - val_loss: 0.9186
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 68s 82ms/step - accuracy: 0.7294 - loss: 0.7945 - val_accuracy: 0.7181 - val_loss: 0.9574
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 70s 82ms/step - accuracy: 0.7426 - loss: 0.7521 - val_accuracy: 0.7565 - val_loss: 0.7709
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 64s 77ms/step - accuracy: 0.7600 - loss: 0.7097 

Saved artifact at '/tmp/tmp9c45jf82'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor_316')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  136800184381328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136800184379984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136800184379216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136800184379600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136800184381136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136800184377872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136800184376720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136800184377104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136800184376912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136800184380752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13680018437